# NeoPZ Python binding with pybind11

### Gustavo Batistela
### 08/10/2019

# What is pybind11?

**pybind11** is a lightweight header-only library that exposes C++ types in Python and vice versa

[https://pybind11.readthedocs.io](https://pybind11.readthedocs.io)


# What is pybind11 capable of binding?

- Functions accepting and returning **custom data structures** per value, reference, or pointer
- Overloaded functions
- Instance methods and static methods and attributes
- Enumerations
- Custom operators
- Single and multiple inheritance
- STL data structures
- And more...


# A simple example


```cpp
#include <pybind11/pybind11.h>

int add(int i, int j) {
    return i + j;
}

PYBIND11_MODULE(example1, m) {
    m.def("add", &add);
}
```

In [25]:
import example1
help(example1.add)
example1.add(1,2)

Help on built-in function add in module example1:

add(...) method of builtins.PyCapsule instance
    add(arg0: int, arg1: int) -> int



3

# A *better documented* simple example


```cpp
#include <pybind11/pybind11.h>

int add(int i, int j) {
    return i + j;
}

PYBIND11_MODULE(example2, m) {
    m.doc() = "A documented example module";
    m.def("add", &add, "A function which adds two numbers",
          py::arg("i"), py::arg("j") = 11);
}
```

In [28]:
import example2
help(example2.add)
example2.add(1, j =2)

Help on built-in function add in module example2:

add(...) method of builtins.PyCapsule instance
    add(i: int, j: int = 11) -> int
    
    A function which adds two numbers



3

# The magic '\_\_repr\_\_' method

```cpp
struct Pet {
    Pet(const std::string &name) : name(name) { }
    void setName(const std::string &name_) { name = name_; }
    const std::string &getName() const { return name; }

    std::string name;
};
```



# The magic '\_\_repr\_\_' method


```cpp
PYBIND11_MODULE(example1, m) {
    m.def("add", &add);
    py::class_<Pet>(m, "Pet")
            .def(py::init<const std::string &>())
            .def("setName", &Pet::setName)
            .def("getName", &Pet::getName);
}
```



In [4]:
import example1 as ex1
a = ex1.Pet("Amora")
print(a)


# The magic '\_\_repr\_\_' method

```cpp
PYBIND11_MODULE(example1, m) {
    py::class_<Pet2>(m, "Pet2", "A Pet class")
        .def(py::init<const std::string &>(), "Pet constructor", py::arg("name"))
        .def("setName", &Pet2::setName, "Sets pet name", py::arg("name"))
        .def("getName", &Pet2::getName, "Reads pet name")
        .def("__repr__", [](const Pet2& pet) {
            std::string r = "Pet2: ";
            r += pet.getName();
            return r;
        }
    );
}
```



In [32]:
import example2 as ex2
a = ex2.Pet2("Amora")
help(example2.Pet2)

Help on class Pet2 in module example2:

class Pet2(pybind11_builtins.pybind11_object)
 |  A Pet class
 |  
 |  Method resolution order:
 |      Pet2
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(...)
 |      __init__(self: example2.Pet2, name: str) -> None
 |      
 |      Pet constructor
 |  
 |  __repr__(...)
 |      __repr__(self: example2.Pet2) -> str
 |  
 |  getName(...)
 |      getName(self: example2.Pet2) -> str
 |      
 |      Reads pet name
 |  
 |  setName(...)
 |      setName(self: example2.Pet2, name: str) -> None
 |      
 |      Sets pet name
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from pybind11_builtins.pybind11_object:
 |  
 |  __new__(*args, **kwargs) from pybind11_builtins.pybind11_type
 |      Create and return a new object.  See help(type) for accurate signature.



# A NeoPZ case
## Constructors

### Original code
```cpp
	/** @brief Constructor of the transformation with square matrix */
	TPZTransform(int dim);
	/** @brief Default constructor */
	TPZTransform();
	/** @brief Constructor of the transformation with rectangular matrix */
	TPZTransform(int fRow,int fCol);
```
### Python binding
```cpp
    py::class_<TPZTransform<double>>(m, "TPZTransform")
        .def(py::init())
        .def(py::init<int>())
        .def(py::init<int, int>())
        
        ...
```
Constructor bindings are generated by `py::init()`

Constructor overloading is done using the different parameters inside `<>`




# A NeoPZ case
## Class methods
### Original code
```cpp
	/** @brief Sets the transformation matrices */
	void SetMatrix(TPZFMatrix<T> &mult,TPZFMatrix<T> &sum);
	
	/** @brief Multiply the transformation object (to the right) with right (Multiplying matrices) */
	TPZTransform<T> Multiply(TPZTransform<T> &right);
	
	/** @brief Transforms the vector */
	void Apply(TPZVec<T> &vectorin,TPZVec<T> &vectorout);
```
### Python binding
```cpp
        .def("SetMatrix", &TPZTransform<double>::SetMatrix)
        .def("Multiply", &TPZTransform<double>::Multiply)
        .def("Apply", &TPZTransform<double>::Apply)
```

In order to bind class methods, a reference to the function needs to be passed

# A NeoPZ case
## Class methods + Lambda expressions
Sometimes it is preferred to define a *lambda expression* instead of passing a reference to an existing function:
```cpp
    // TPZManVector<double> bindings
    py::class_<TPZManVector<double>>(m, "TPZVecDouble")
        
        ...
        
        // Lambda needed for index verification
        .def("__getitem__",
             [](const TPZManVector<double>& vec, int64_t position) {
                 if (position >= vec.size() || position < 0) throw py::index_error();
                 return vec[position];
             },
             py::is_operator()
        )
        // Lambda needed since print function returning a std::string object does not exist in NeoPZ
        .def("__repr__",
            [](TPZManVector<double>& vec) {
               std::stringstream repr;
               vec.Print(repr);
               return repr.str();
            }
        )
        
        ...
```

**Note that this way of defining the *\_\_repr\_\_* method is very convenient.**

**It reuses NeoPZ code and will not change across different classes since they all implement a Print(std::ostream) method.**


# A NeoPZ case
## Overloading methods
### Original code
```cpp
		/** @brief Get all sides with lower dimension on side */	
		static void LowerDimensionSides(int side,TPZStack<int> &smallsides);
		/** @brief Get all sides with lower dimension but equal to DimTarget on side */
		static void LowerDimensionSides(int side,TPZStack<int> &smallsides, int DimTarget);
```
### Python binding
```cpp
    // TPZPrism bindings
    py::class_<pztopology::TPZPrism>(m, "TPZPrism")
        
        ...
        
        .def_static("LowerDimensionSides",
            py::overload_cast<int, TPZStack<int> &>(&pztopology::TPZPrism::LowerDimensionSides))
        .def_static("LowerDimensionSides", 
            py::overload_cast<int, TPZStack<int> &, int>(&pztopology::TPZPrism::LowerDimensionSides))
        
        ...
```
Class methods are overloaded using `py::overload_cast<>` and declaring the different parameters inside `<>`.

Static methods are generated by using `.def_static` instead of `.def` (nearly all Topology methods are bound this way).

# A NeoPZ case
## Overloading methods II
### Original code
```cpp
	const TPZFMatrix<T>  & Mult() const {return fMult;}
	
	TPZFMatrix<T>  & Mult() {return fMult;}
```
### Python binding
```cpp
        .def("Mult", py::overload_cast<>(&TPZTransform<double>::Mult))
        .def("Mult", py::overload_cast<>(&TPZTransform<double>::Mult, py::const_))
```
Methods that are overloaded by *constness* require `py::const_` at the end.

# A NeoPZ case
## Class members

It's possible to "*bind*" class members by passing a reference to the getter and setter function.

```cpp
        .def_property_readonly("fMult", py::overload_cast<>(&TPZTransform<double>::Mult))
        .def_property_readonly("fSum", py::overload_cast<>(&TPZTransform<double>::Sum))
```

This makes possible to call:
```python
    a = TPZTransform.fMult
```
Instead of
```python
    a = TPZTransform.Mult()
```

Not much of a difference though.

In [34]:
from neopz import *

print("Testing quadrilateral NumSides and NSideNodes\n")
for sideId in range(TPZTriangle.NumSides()):
    print(TPZTriangle.NSideNodes(sideId))


Testing quadrilateral NumSides and NSideNodes

1
1
1
2
2
2
3


In [35]:
from neopz import *


trans = TPZTriangle.SideToSideTransform(TPZTriangle.NumSides() - 1, 3)
print(trans)


TPZTransform
  Mult '(1 x 2)' = [
	2.000000  0.000000  
]
  Sum'(1 x 1)' = [
	-1.000000  
]


In [8]:
from neopz import *

print("\nTesting container classes\n")

matrix = TPZMatrix(2, 1, 1.)
print(matrix)

matrix = TPZTetrahedron.SideToSideTransform(TPZTetrahedron.NumSides() - 1, 9).Mult()
print(matrix)

vec = TPZVecDouble(3, 1.)
print(vec)



Testing container classes

TPZMatrix '(2 x 1)' = [
	1.000000  
	1.000000  
]
TPZMatrix '(1 x 3)' = [
	0.000000  -1.000000  1.000000  
]

Number of elements = 3


# An important remark

You don't need to implement *every* method of a class to make it visible in python.

In fact you don't need **any** method to be implemented.
\
\
\
If you are only interested in returning an object from a first method and then passing as argument to a second method, the following line will suffice:

```cpp
py::class_<TPZStack<int>>(m, "TPZStackInt");
```

If somehow the object also needs to be **initialized** in Python, you'll aditionally have to bind one constructor. But nothing more than that...
```cpp
py::class_<TPZStack<int>>(m, "TPZStackInt")
    .def(py::init());
```

# An important remark

Alright, alright, maybe being able to print the object would be nice:
```cpp
    .def("__repr__",
        [](const TPZStack<int>& stack) {
            std::string r("TPZStackInt [");
            for (int i = 0; i < stack.NElements(); i++) {
                r += std::to_string(stack[i]);
                if (i != stack.NElements() - 1) {
                    r += ", ";
                }
            }
            r += "]";
            return r;
        }
    )
```

In this example I didn't use the Print method available in NeoPZ. As I said before, _very inconvenient_...


In [24]:
from neopz import *

stack = TPZStackInt()
print(stack)
TPZTriangle.LowerDimensionSides(6, stack)
print(stack)


TPZStackInt []
TPZStackInt [0, 1, 2, 3, 4, 5]


# Obrigado!